# Testing
### Durchschnittliches Alter bei der Erstheirat, nach Geschlecht, seit 1993
Datum: 02.11.2021

**Importiere die notwendigen Packages**

In [1]:
#pip install geopandas fiona requests folium mplleaflet contextily

In [2]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 



Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [3]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

Importiere einen Datensatz von der **DWH_INT** Dropzone.

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset anschauen auf INTEG Datenkatalog**: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_ehe_avg-alter_geschlecht_od3302

(int_dwh)

In [19]:
#Die Datasets sind nur zum Testen auf INT-DWH-Dropzone. Wenn der Test vorbei ist, sind sie auf PROD. 
# Über den Status kann man einfach switchen

status = "prod";
print(status)

prod


In [ ]:
# Filepath
if status == "prod":
    fp = r"\\szh\ssz\applikationen\OGD_Dropzone\DWH\bev_ehe_avg-alter_geschlecht_OD3302\BEV330OD3302.csv"
else:
    fp = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bev_ehe_avg-alter_geschlecht_OD3302\BEV330OD3302.csv"

In [12]:
# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )


In der Folge ein paar erste Tests:

In [13]:
type(data2betested)

pandas.core.frame.DataFrame

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [14]:
data2betested.head(3)

,StichtagDatJahr,SexLang,SexCd,DurchAlter
0,1993,weiblich,2,28
1,1993,männlich,1,30
2,1994,weiblich,2,27


In [15]:
data2betested.dtypes

StichtagDatJahr      int64
SexLang             object
SexCd                int64
DurchAlter         float64
dtype: object

In [16]:
data2betested.shape

(56, 4)

Beschreibe einzelne Attribute

In [ ]:
data2betested.describe()

Wie viele Nullwerte gibt es im Datensatz?

In [ ]:
data2betested.isnull().sum()

Wandle die Spalte Jahr von Int zu **Datetime** um. Dies wird in den meisten Grafiken so benötigt.

In [ ]:
data2betested['StichtagDatJahr']=pd.to_datetime(data2betested['StichtagDatJahr'], format='%Y')

#TO DO --> setze das Datum auf den 31.12. der Jahres.
#test=data2betested[['StichtagDatJahr']].date().replace(month=12, day=31)

data2betested

Bereite die Daten für die Grafik unten vor

In [ ]:
Jahre = data2betested.groupby("StichtagDatJahr")
len(Jahre)

In [ ]:
for key, group in Jahre:
    print(key, len(group))

In [ ]:
myJahr = "2020-01-01"
aktuellesJahr=Jahre.get_group(myJahr)   
aktuellesJahr

**Selektionen und Aggregationen**

Check this [Doku](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [ ]:
#test = data2betested.groupby(['StichtagDatJahr', 'AlterV20Kurz'])[['AnzBestWir']].agg()

# produces Pandas Series
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])['AnzBestWir'].sum()
# Produces Pandas DataFrame
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])[['AnzBestWir']].sum()
    

**Einfache Visualisierungen zur Plausi**

Exploriere die Daten mit Pivottable.JS

In [ ]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [ ]:
data2betested.head()

In [ ]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]
#mySelection[[mySelection.NationFHistLang =='Schweiz']]


highlight = alt.selection(type='single', on='mouseover',
                          fields=['SexLang'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(data2betested).encode(
    x='StichtagDatJahr',
    y='DurchAlter',
    color='SexLang'
)

points = base.mark_circle().encode(
    opacity=alt.value(2)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

points + lines

In [ ]:
alt.Chart(data2betested).mark_circle(size=60).encode(
    x='StichtagDatJahr',
    y='DurchAlter',
    #color='AnzNat',
    #color=alt.Color('AnzNat', scale=alt.Scale(scheme='greens')),
    color=alt.Color('SexLang', scale=alt.Scale(scheme='dark2')),
    tooltip=['StichtagDatJahr','SexLang','DurchAlter']
).interactive() # this makes the axes interactive: now you can zoom & pan




**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](http://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/EditForm.aspx?ID=52&Source=%2Forga%2Fssz%2Dprodukte%2FLists%2FSASA%5FOutputs)**